In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import zipfile

# Unzip the data folder first

In [2]:
#hc - housing core
#bls - bureal of labor
#ctr - corporate tax rate
#rm -  rent median
#cs - census_starter
#fa - food access
#ue - unemployment
#me - medicare enrollment

df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv')
df_bls = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.xlsx')
df_ctr = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Corporate_tax_rate.csv')
df_rm_FY19 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2019_50_County_rev.xlsx')
df_rm_FY20 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2020_50_County_rev.xlsx')
df_rm_FY21 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2021_50_County.xlsx')
df_rm_FY22 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2022_FMR_50_county_rev.xlsx')
df_rm_FY23 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2023_FMR_50_county.xlsx')
df_fa = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FoodAccessResearchAtlasData2019.xlsx')
df_ue = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\Unemployment.xlsx', skiprows=4) #Bianca
df_me = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Medicare_Monthly_Enrollment_September_2022.csv')


df_cs = pd.read_csv('..\Data\data_zip\census_starter.csv')
df_test = pd.read_csv(r'..\Data\data_zip\test.csv')
df_train = pd.read_csv(r'..\Data\data_zip\train.csv')

C:\Users\jacks\AppData\Local\Temp\ipykernel_45388\736578475.py:10: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv')


## Format Census Starter to join into train and test data

In [3]:
df_cs
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
0,1001,2017,55317.0,76.6,14.5,2.1,1.3
1,1001,2018,58786.0,78.9,15.9,2.0,1.1
2,1001,2019,58731.0,80.6,16.1,2.3,0.7
3,1001,2020,57982.0,82.7,16.7,2.3,0.6
4,1001,2021,62660.0,85.5,16.4,2.1,1.1
...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0


In [4]:
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
df_cs2.sort_values(['cfips']).reset_index(drop=True)
df_cs2 = pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_cs2

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
0,1001,2017,55317.0,76.6,14.5,2.1,1.3
1,1001,2018,58786.0,78.9,15.9,2.0,1.1
2,1001,2019,58731.0,80.6,16.1,2.3,0.7
3,1001,2020,57982.0,82.7,16.7,2.3,0.6
4,1001,2021,62660.0,85.5,16.4,2.1,1.1
...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0


### df_hc housing core looks like it is already in a joinable format

## Format bureal of labor to join into train and test data

In [5]:
df_bls2 = pd.pivot_table(df_bls, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2

measure_type,cfips,first_day_of_month,Employment,Labor force,Unemployment,Unemployment rate
0,1001,2011-01-01,23440.0,25800.0,2360.0,9.1
1,1001,2011-02-01,23516.0,25850.0,2334.0,9.0
2,1001,2011-03-01,23643.0,25862.0,2219.0,8.6
3,1001,2011-04-01,23787.0,25872.0,2085.0,8.1
4,1001,2011-05-01,23773.0,25863.0,2090.0,8.1
...,...,...,...,...,...,...
87646,17045,2022-06-01,NaN,NaN,318.0,3.6
87647,17045,2022-07-01,NaN,NaN,352.0,3.8
87648,17045,2022-08-01,NaN,NaN,327.0,3.6
87649,17045,2022-09-01,NaN,NaN,281.0,3.2


In [6]:
df_ctr2 = pd.melt(df_ctr, id_vars=['State'])
df_ctr2

,State,variable,value
0,Alabama,2008,4.23%
1,Alaska,2008,9.40%
2,Arizona,2008,6.97%
3,Arkansas,2008,6.50%
4,California,2008,8.84%
...,...,...,...
745,Virginia,2022,7.63%
746,Washington,2022,6.81%
747,West Virginia,2022,6.50%
748,Wisconsin,2022,7.90%


In [7]:
df_me
#No formatting change needed

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
0,2022,April,National,US,National,Total,NaN,64665615,35102417,29563198,...,56614798,7752840,236979,7515861,58785851,29255529,29530322,50013581,23367558,26646023
1,2022,April,State,AL,Alabama,Total,1.0,1078349,495600,582749,...,876402,196610,5000,191610,998869,416218,582651,822177,268695,553482
2,2022,April,County,AL,Alabama,Autauga,1001.0,11492,5150,6342,...,9099,2339,55,2284,10643,4303,6340,7985,1829,6156
3,2022,April,County,AL,Alabama,Baldwin,1003.0,58902,27624,31278,...,51904,6798,131,6667,55097,23830,31267,44841,15131,29710
4,2022,April,County,AL,Alabama,Barbour,1005.0,6260,2440,3820,...,4877,1329,41,1288,5932,2113,3819,5050,1396,3654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,2021,September,County,VI,Virgin Islands,St. John,78020.0,921,745,176,...,*,47,*,*,838,662,176,470,299,171
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030.0,9625,6669,2956,...,8883,669,50,619,8675,5719,2956,5792,2859,2933
60038,2021,September,County,VI,Virgin Islands,Unknown,78999.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*
60039,2021,September,State,UK,Unknown,Unknown,99.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [8]:
df_ue2 = pd.melt(df_ue, id_vars=['FIPS_code', 'State', 'Area_name'])
df_ue2['Year'] = df_ue2['variable'].str[-4:]
df_ue2['variable'] = df_ue2['variable'].str[:-5]
df_ue2 = pd.pivot_table(df_ue2, values='value', index=['FIPS_code', 'State', 'Area_name', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_ue2['Year'].max()

'2021'

Unemployment data on a two year lag. Match the 2021 data with 2023 year, 2020 with 2022 and so on

In [9]:
df_me

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
0,2022,April,National,US,National,Total,NaN,64665615,35102417,29563198,...,56614798,7752840,236979,7515861,58785851,29255529,29530322,50013581,23367558,26646023
1,2022,April,State,AL,Alabama,Total,1.0,1078349,495600,582749,...,876402,196610,5000,191610,998869,416218,582651,822177,268695,553482
2,2022,April,County,AL,Alabama,Autauga,1001.0,11492,5150,6342,...,9099,2339,55,2284,10643,4303,6340,7985,1829,6156
3,2022,April,County,AL,Alabama,Baldwin,1003.0,58902,27624,31278,...,51904,6798,131,6667,55097,23830,31267,44841,15131,29710
4,2022,April,County,AL,Alabama,Barbour,1005.0,6260,2440,3820,...,4877,1329,41,1288,5932,2113,3819,5050,1396,3654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,2021,September,County,VI,Virgin Islands,St. John,78020.0,921,745,176,...,*,47,*,*,838,662,176,470,299,171
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030.0,9625,6669,2956,...,8883,669,50,619,8675,5719,2956,5792,2859,2933
60038,2021,September,County,VI,Virgin Islands,Unknown,78999.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*
60039,2021,September,State,UK,Unknown,Unknown,99.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [10]:
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [11]:
df_train['first_day_of_month'] = pd.to_datetime(df_train['first_day_of_month'])

In [12]:
df_bls2['first_day_of_month_lag'] = df_bls2['first_day_of_month'] + pd.DateOffset(months=12)

In [13]:
print(df_bls2.dtypes)
print(df_train.dtypes)

measure_type
cfips                              int64
first_day_of_month        datetime64[ns]
Employment                       float64
Labor force                      float64
Unemployment                     float64
Unemployment rate                float64
first_day_of_month_lag    datetime64[ns]
dtype: object
row_id                           object
cfips                             int64
county                           object
state                            object
first_day_of_month       datetime64[ns]
microbusiness_density           float64
active                            int64
dtype: object


In [14]:
df_train_bls = df_train.merge(df_bls2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag'], indicator=True)

In [15]:
df_bls2['first_day_of_month'].max()

Timestamp('2022-11-01 00:00:00')

In [16]:
df_train_bls

,row_id,cfips,county,state,first_day_of_month_x,microbusiness_density,active,first_day_of_month_y,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2018-08-01,25484.0,26496.0,1012.0,3.8,2019-08-01,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2018-09-01,25443.0,26354.0,911.0,3.5,2019-09-01,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2018-10-01,25614.0,26526.0,912.0,3.4,2019-10-01,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2018-11-01,25639.0,26485.0,846.0,3.2,2019-11-01,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2018-12-01,25664.0,26543.0,879.0,3.3,2019-12-01,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only


In [17]:
df_train_bls[df_train_bls['Employment'].isnull()]

,row_id,cfips,county,state,first_day_of_month_x,microbusiness_density,active,first_day_of_month_y,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag,_merge
21294,15005_2019-08-01,15005,Kalawao County,Hawaii,2019-08-01,8.333333,7,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21295,15005_2019-09-01,15005,Kalawao County,Hawaii,2019-09-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21296,15005_2019-10-01,15005,Kalawao County,Hawaii,2019-10-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21297,15005_2019-11-01,15005,Kalawao County,Hawaii,2019-11-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21298,15005_2019-12-01,15005,Kalawao County,Hawaii,2019-12-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only


In [18]:
df_test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [19]:
bls_test = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.csv')

In [20]:
bls_test

,series_id,year,period,value,footnote_codes,cfips,first_day_of_month,measure_type
0,LAUCN010010000000003,1990,M01,6.5,NaN,1001,1990-01-01,Unemployment rate
1,LAUCN010010000000003,1990,M02,6.5,NaN,1001,1990-02-01,Unemployment rate
2,LAUCN010010000000003,1990,M03,5.7,NaN,1001,1990-03-01,Unemployment rate
3,LAUCN010010000000003,1990,M04,6.6,NaN,1001,1990-04-01,Unemployment rate
4,LAUCN010010000000003,1990,M05,6.0,NaN,1001,1990-05-01,Unemployment rate
...,...,...,...,...,...,...,...,...
5483039,LAUCN721530000000006,2022,M06,10162.0,NaN,72153,2022-06-01,Labor force
5483040,LAUCN721530000000006,2022,M07,9987.0,NaN,72153,2022-07-01,Labor force
5483041,LAUCN721530000000006,2022,M08,10276.0,P,72153,2022-08-01,Labor force
5483042,LAUCN721530000000006,2022,M09,NaN,N,72153,2022-09-01,Labor force


In [21]:
bls_test['cfips'].unique()


array([ 1001,  1003,  1005, ..., 72149, 72151, 72153], dtype=int64)

In [23]:
df_bls2 = bls_test[bls_test['period'] != 'M13']
#df_bls2['cfips'] = df_bls2['cfips'].astype('object')

In [24]:
df_bls2_2 = pd.pivot_table(df_bls2, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2_2

measure_type,cfips,first_day_of_month,Employment,Labor force,Unemployment,Unemployment rate
0,1001,1990-01-01,15504.0,16582.0,1078.0,6.5
1,1001,1990-02-01,15515.0,16591.0,1076.0,6.5
2,1001,1990-03-01,15768.0,16719.0,951.0,5.7
3,1001,1990-04-01,15787.0,16898.0,1111.0,6.6
4,1001,1990-05-01,15869.0,16889.0,1020.0,6.0
...,...,...,...,...,...,...
1267790,72153,2022-06-01,9202.0,10162.0,960.0,9.4
1267791,72153,2022-07-01,9090.0,9987.0,897.0,9.0
1267792,72153,2022-08-01,9309.0,10276.0,967.0,9.4
1267793,72153,2022-09-01,0.0,0.0,0.0,0.0


In [25]:
df_bls2_2['first_day_of_month'] = pd.to_datetime(df_bls2_2['first_day_of_month'])

In [26]:
df_bls2_2['first_day_of_month_lag_6'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=6)
df_bls2_2['first_day_of_month_lag_7'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=7)
df_bls2_2['first_day_of_month_lag_8'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=8)
df_bls2_2['first_day_of_month_lag_9'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=9)
df_bls2_2['first_day_of_month_lag_10'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=10)
df_bls2_2['first_day_of_month_lag_11'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=11)
df_bls2_2['first_day_of_month_lag_12'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=12)

In [27]:
df_train_bls6 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_6'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls6 = df_train_bls6[df_train_bls6['Employment'].notnull()]

df_train_bls7 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_7'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls7 = df_train_bls7[df_train_bls7['Employment'].notnull()]

df_train_bls8 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_8'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls8 = df_train_bls8[df_train_bls8['Employment'].notnull()]

df_train_bls9 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_9'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls9 = df_train_bls9[df_train_bls9['Employment'].notnull()]

df_train_bls10 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_10'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls10 = df_train_bls10[df_train_bls10['Employment'].notnull()]

df_train_bls11 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_11'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls11 = df_train_bls11[df_train_bls11['Employment'].notnull()]

df_train_bls12 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_12'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls12 = df_train_bls12[df_train_bls12['Employment'].notnull()]

In [28]:
df_train_bls6

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,first_day_of_month_bls,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag_6,first_day_of_month_lag_7,first_day_of_month_lag_8,first_day_of_month_lag_9,first_day_of_month_lag_10,first_day_of_month_lag_11,first_day_of_month_lag_12,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019-02-01,25684.0,26590.0,906.0,3.4,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019-03-01,25742.0,26608.0,866.0,3.3,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019-04-01,25969.0,26627.0,658.0,2.5,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019-05-01,26160.0,26792.0,632.0,2.4,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019-06-01,26042.0,26858.0,816.0,3.0,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2021-12-01,3654.0,3742.0,88.0,2.4,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,both
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022-01-01,3631.0,3740.0,109.0,2.9,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,both
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022-02-01,3621.0,3735.0,114.0,3.1,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,both
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022-03-01,3606.0,3722.0,116.0,3.1,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,both


In [29]:
#6 month lag seems like the best correlations
df_train_bls6[['microbusiness_density', 'Employment', 'Labor force', 'Unemployment', 'Unemployment rate']].corr()

,microbusiness_density,Employment,Labor force,Unemployment,Unemployment rate
microbusiness_density,1.000000,0.278992,0.276131,0.196738,-0.018429
Employment,0.278992,1.000000,0.999054,0.825855,0.046060
Labor force,0.276131,0.999054,1.000000,0.849600,0.058305
Unemployment,0.196738,0.825855,0.849600,1.000000,0.197360
Unemployment rate,-0.018429,0.046060,0.058305,0.197360,1.000000


In [32]:
import statsmodels.api as sm
from itertools import combinations

vars_list = ['Employment', 'Labor force', 'Unemployment', 'Unemployment rate']
vars_combinations = list(combinations(vars_list, 1)) + list(combinations(vars_list, 2)) + list(combinations(vars_list, 3)) + list(combinations(vars_list, 4))
vars_combinations = [list(tup) for tup in vars_combinations]
print(vars_combinations)

for lists in vars_combinations:
    x = df_train_bls10[lists]
    y = df_train_bls10[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

[['Employment'], ['Labor force'], ['Unemployment'], ['Unemployment rate'], ['Employment', 'Labor force'], ['Employment', 'Unemployment'], ['Employment', 'Unemployment rate'], ['Labor force', 'Unemployment'], ['Labor force', 'Unemployment rate'], ['Unemployment', 'Unemployment rate'], ['Employment', 'Labor force', 'Unemployment'], ['Employment', 'Labor force', 'Unemployment rate'], ['Employment', 'Unemployment', 'Unemployment rate'], ['Labor force', 'Unemployment', 'Unemployment rate'], ['Employment', 'Labor force', 'Unemployment', 'Unemployment rate']]
                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.078
Model:                               OLS   Adj. R-squared:                  0.078
Method:                    Least Squares   F-statistic:                 1.030e+04
Date:                   Thu, 02 Feb 2023   Prob (F-statistic):               0.00
Time:                          

From these OLS Regression results we have decided to keep Employment, Unemployment, and Unemployment rate